In [1]:
from image import *

In [4]:
# RGB image
preprocess = {"nothing": lambda x: x, "contrast": increase_contrast, "normalize": normalize, "blackhat": blackhat}
hairs = {"nothing": lambda x: x, "median": median, "low_pass_filter": low_pass_filter_rgb, "dilate": dilate}
segments = {"threshold": threshold, "watershed": watershed, "mser": mser, "k_means": k_means}

img_folder = "images/original"
seg_folder = "images/segmented"

results = {}

for pre in preprocess:
    for hair in hairs:
        for seg in segments:

            name = "{} - {} - {}".format(pre, hair, seg)
            results[name] = {}
            avg_TPR = 0
            avg_FPR = 0
            avg_TP = 0
            avg_TN = 0
            avg_FP = 0
            avg_FN = 0
            cont = 0

            for img_name in sorted(os.listdir(img_folder)):
                cont += 1
                aux_index = img_name.index("i")
                n = img_name[0:aux_index]
                # Real image
                img_path = img_folder + "/" + img_name
                img = load_image_rgb(img_path)

                # Segmented by function
                func_seg_img = segments[seg](hairs[hair](preprocess[pre](img)))

                # Improve the segmentation by applying morphology operations
                opening = cv2.morphologyEx(func_seg_img, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
                final = cv2.morphologyEx(opening, cv2.MORPH_DILATE, np.ones((5,5),np.uint8),iterations=3)
                final = cv2.medianBlur(final,9)
                final = cv2.morphologyEx(final, cv2.MORPH_DILATE, np.ones((7,7),np.uint8),iterations=3)
                final = cv2.medianBlur(final,11)
                func_seg_img = cv2.morphologyEx(final, cv2.MORPH_CLOSE, np.ones((7,7),np.uint8))

                # Real segmentation
                real_seg_path = seg_folder + "/" + n + "seg.png"
                real_seg_img = load_image(real_seg_path)

                # Compare
                TPR, FPR, TP, TN, FP, FN = compare_seg(real_seg_img, func_seg_img)
                this_res = {
                    "TP": TP,
                    "FP": FP,
                    "TN": TN,
                    "FN": FN,
                    "TPR": TPR,
                    "FPR": FPR,
                }
                results[name][n] = this_res
                avg_TPR += TPR
                avg_FPR += FPR
                avg_TP += TP
                avg_TN += TN
                avg_FP += FP
                avg_FN += FN

            results[name]["average_TPR"] = avg_TPR/cont
            results[name]["average_FPR"] = avg_FPR/cont
            results[name]["average_TP"] = avg_TP/cont
            results[name]["average_TN"] = avg_TN/cont
            results[name]["average_FP"] = avg_FP/cont
            results[name]["average_FN"] = avg_FN/cont

            print(name)
            print("Avg TPR:", results[name]["average_TPR"])
            print("Avg FPR:", results[name]["average_FPR"])
            print()       



nothing - nothing - threshold
Avg TPR: 0.9467833069495523
Avg FPR: 0.2044462853793481

nothing - nothing - watershed
Avg TPR: 0.927195485892116
Avg FPR: 0.47131384101562396

nothing - nothing - mser
Avg TPR: 0.1493562367326172
Avg FPR: 0.06102014404717992

nothing - nothing - k_means
Avg TPR: 0.964958948601586
Avg FPR: 0.20405029281444675

nothing - median - threshold
Avg TPR: 0.9215622851343842
Avg FPR: 0.14576848497391112

nothing - median - watershed
Avg TPR: 0.4064092787770654
Avg FPR: 0.05404133896194641

nothing - median - mser
Avg TPR: 0.12313684422817842
Avg FPR: 0.0075165475896958375

nothing - median - k_means
Avg TPR: 0.9455422102197866
Avg FPR: 0.14084598250050928

nothing - low_pass_filter - threshold
Avg TPR: 0.9271735265706357
Avg FPR: 0.15279052467023793

nothing - low_pass_filter - watershed
Avg TPR: 0.3969314857497135
Avg FPR: 0.08235186992023573

nothing - low_pass_filter - mser
Avg TPR: 0.08928256064810086
Avg FPR: 0.007061954195144517

nothing - low_pass_filter - k